In [1]:
import json
import pandas as pd

# Create our table index from rspamd extracted data

In [2]:
df = pd.read_csv("generated/rspamd.csv")[["email_sent_to", 'targeted_esp']].drop_duplicates()
# Remove the test-control domain
df = df[df["email_sent_to"] != "control-domain.example" ]
# Rename collumns
df = df.rename(columns={"email_sent_to": "esp_name", "targeted_esp": "esp_label"})
df

,esp_name,esp_label
0,zohomail.eu,zohomaileu
1,yandex.com,yandexcom
2,yahoo.com,yahoocom
3,seznam.cz,seznamcz
4,sapo.pt,sapopt
5,rediff.com,rediffcom
6,protonmail.com,protonmailcom
7,orange.fr,orangefr
8,op.pl,oppl
9,mailfence.com,mailfencecom


# Add the DNS log analysis

In [3]:
# Join bind Database
dns_records_dataframe = pd.read_csv("generated/bind_results.csv")
df = df.set_index("esp_label").join(dns_records_dataframe.set_index('ESP'))
df

,esp_name,SPF_txt,SPF_exsit,DKIM_txt,DMARC_txt,EDV_process
esp_label,,,,,,
zohomaileu,zohomail.eu,True,True,True,True,False
yandexcom,yandex.com,True,False,True,True,False
yahoocom,yahoo.com,True,True,True,True,True
seznamcz,seznam.cz,True,True,True,True,False
sapopt,sapo.pt,True,True,True,True,False
rediffcom,rediff.com,True,True,True,True,False
protonmailcom,protonmail.com,True,True,True,True,False
orangefr,orange.fr,True,True,True,True,False
oppl,op.pl,True,True,True,True,True


# Add the Authentication-Results headers informations

In [4]:
with open("generated/auth_info.json", "r") as fp:
    header_info = json.load(fp)

def extract_header_info(esp, header_name):
    esp_data = header_info.get(esp)
    if esp_data is None:
        return None
    return header_name in esp_data
    
df["dmarc_header"] = df.index.map(lambda esp: extract_header_info(esp, "dmarc"))
df["dkim_header"] = df.index.map(lambda esp: extract_header_info(esp, "dkim"))
df["spf_header"] = df.index.map(lambda esp: extract_header_info(esp, "spf"))

df

,esp_name,SPF_txt,SPF_exsit,DKIM_txt,DMARC_txt,EDV_process,dmarc_header,dkim_header,spf_header
esp_label,,,,,,,,,
zohomaileu,zohomail.eu,True,True,True,True,False,True,True,True
yandexcom,yandex.com,True,False,True,True,False,False,True,True
yahoocom,yahoo.com,True,True,True,True,True,True,True,True
seznamcz,seznam.cz,True,True,True,True,False,False,False,False
sapopt,sapo.pt,True,True,True,True,False,False,False,False
rediffcom,rediff.com,True,True,True,True,False,True,True,True
protonmailcom,protonmail.com,True,True,True,True,False,True,True,True
orangefr,orange.fr,True,True,True,True,False,False,True,False
oppl,op.pl,True,True,True,True,True,True,True,True


# Add the DMARC aggregate report test results 

In [5]:
with open("generated/reporting_feature", 'r') as fp:
    reporting_features = json.load(fp)

df["rua"] = df["esp_name"].apply(lambda x : x in reporting_features["esp_sending_rua"])
df["ruf"] = df["esp_name"].apply(lambda x : x in reporting_features["esp_sending_ruf"])
df["rua_with_edv_success"] = df["esp_name"].apply(lambda x : x in reporting_features["esp_sending_rua"] if x in reporting_features["esp_sending_rua"] else None)
df["rua_with_edv_failure"] = df["esp_name"].apply(lambda x : not x in reporting_features["esp_sending_rua_edv_failed"] if x in reporting_features["esp_sending_rua"] else None)
df["rua_with_rewritting_edv"] = df["esp_name"].apply(lambda x : x in reporting_features["edv_rewritting_rua"] if x in reporting_features["esp_sending_rua"] else None)


df = df.sort_values(by=['esp_name'], ascending=True)
df

,esp_name,SPF_txt,SPF_exsit,DKIM_txt,DMARC_txt,EDV_process,dmarc_header,dkim_header,spf_header,rua,ruf,rua_with_edv_success,rua_with_edv_failure,rua_with_rewritting_edv
esp_label,,,,,,,,,,,,,,
163com,163.com,True,False,True,True,False,False,True,True,False,False,None,None,None
fastmailcom,fastmail.com,True,True,True,True,True,True,True,True,True,False,True,True,True
freemailhu,freemail.hu,False,False,False,False,False,False,False,False,False,False,None,None,None
gmailcom,gmail.com,True,True,True,True,False,True,True,True,True,False,True,False,False
gmxnet,gmx.net,True,True,True,True,False,False,True,False,False,False,None,None,None
heycom,hey.com,True,True,True,True,False,True,True,True,False,False,None,None,None
inboxlv,inbox.lv,True,True,True,True,False,True,True,True,False,False,None,None,None
interiapl,interia.pl,True,True,True,True,False,False,False,False,False,False,None,None,None
kolabnowcom,kolabnow.com,True,True,True,False,False,False,True,False,False,False,None,None,None


In [6]:
table_data = json.loads(df.to_json())
table_key = list(table_data["esp_name"].keys())
print(table_key)
print(table_data.keys())

['163com', 'fastmailcom', 'freemailhu', 'gmailcom', 'gmxnet', 'heycom', 'inboxlv', 'interiapl', 'kolabnowcom', 'lapostenet', 'mailcom', 'mailru', 'mailfencecom', 'navercom', 'oppl', 'orangefr', 'outlookcom', 'protonmailcom', 'rediffcom', 'sapopt', 'seznamcz', 'tutanotacom', 'yahoocom', 'yandexcom', 'zohomaileu']
dict_keys(['esp_name', 'SPF_txt', 'SPF_exsit', 'DKIM_txt', 'DMARC_txt', 'EDV_process', 'dmarc_header', 'dkim_header', 'spf_header', 'rua', 'ruf', 'rua_with_edv_success', 'rua_with_edv_failure', 'rua_with_rewritting_edv'])


# Generate the LaTeX table from the datasets

In [7]:
def build_table(structure, table_data, builded_struct ="", translate={}, line_break_keys=[]):
    res = "% Python Script beginning here\n"
    if builded_struct == "":
        builded_struct = "l|"
        for i in range(0, len(table_key)):
            builded_struct += "b{0.1cm}|"

        builded_struct = builded_struct[:-1]

    res +=f"\\begin{{tabular}}{{{builded_struct}}}\n\n"
    f = 0
   
    res += "Features &"
    for k in table_key:
        res += ' \\begin{turn}{90}' + f'{table_data["esp_name"][k]}' + '\\end{turn}  &'
        
    res = res[:-1] + "\\\\ \\specialrule{1pt}{0pt}{0pt}  \n"

    feature_no = 1
    for f in structure:
        res += f' $F_{{{feature_no}}}$ : {translate.get(f, f)} &'
        feature_no += 1
        for k in table_key:
            res += f' {table_data[f][k]} &'
        
        res = res[:-1] + ' \\\\[1ex] \n'
        
        if f in line_break_keys:
            res += '\specialrule{0.5pt}{0pt}{0pt}  \n'
       
    
    res +="\n\\specialrule{1pt}{0pt}{0pt} \n\\end{tabular}\n"
    res +=  "% Python Script ending here\n"
    return res



trad_tex = {
    'SPF_txt':'$SPF_{record_{txt}}$', 
    'SPF_exsit':'$SPF_{record_{a}}$', 
    'DKIM_txt':'$DKIM_{record}$', 
    'DMARC_txt':'$DMARC_{record}$', 
    'EDV_process':'$EDV_{query}$', 
    'dmarc_header':'$DMARC_{header}$', 
    'dkim_header':'$DKIM_{header}$', 
    'spf_header':'$SPF_{header}', 
    'rua':'$DMARC_{rua}$', 
    'ruf':"$DMARC_{ruf}$", 
    'rua_with_edv_success':'$EDV_{control}$', 
    'rua_with_edv_failure':'$EDV_{failure}$', 
    'rua_with_rewritting_edv':'$EDV_{overwrite}$'
    
}


# december_sumbission_structure = ['SPF_txt', 'SPF_exsit', 'DKIM_txt', 'DMARC_txt', 'spf_header', 'dkim_header', 'dmarc_header'  , 'rua', 'ruf', 'EDV_process', 'rua_with_edv_success', 'rua_with_edv_failure', 'rua_with_rewritting_edv']

structure = ['DMARC_txt', 'dmarc_header'  , 'rua', 'ruf', 'EDV_process', 'rua_with_edv_success', 'rua_with_edv_failure', 'rua_with_rewritting_edv']


latex_table = build_table(structure=structure, table_data=table_data, translate=trad_tex, line_break_keys=["DMARC_txt", "dmarc_header", "ruf"])
latex_table = latex_table.replace("True", "{\\color{OliveGreen}\\checkmark}")
latex_table = latex_table.replace("False", "{\\color{red}\\ding{55}}")
latex_table = latex_table.replace("None", "-")
print(latex_table)

with open("dmarc_features_table.tex", 'w') as fp:
    fp.write(latex_table)



% Python Script beginning here
\begin{tabular}{l|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}|b{0.1cm}}

Features & \begin{turn}{90}163.com\end{turn}  & \begin{turn}{90}fastmail.com\end{turn}  & \begin{turn}{90}freemail.hu\end{turn}  & \begin{turn}{90}gmail.com\end{turn}  & \begin{turn}{90}gmx.net\end{turn}  & \begin{turn}{90}hey.com\end{turn}  & \begin{turn}{90}inbox.lv\end{turn}  & \begin{turn}{90}interia.pl\end{turn}  & \begin{turn}{90}kolabnow.com\end{turn}  & \begin{turn}{90}laposte.net\end{turn}  & \begin{turn}{90}mail.com\end{turn}  & \begin{turn}{90}mail.ru\end{turn}  & \begin{turn}{90}mailfence.com\end{turn}  & \begin{turn}{90}naver.com\end{turn}  & \begin{turn}{90}op.pl\end{turn}  & \begin{turn}{90}orange.fr\end{turn}  & \begin{turn}{90}outlook.com\end{turn}  & \begin{turn}{90}protonmail.com\end{turn}  & \be